In [1]:
import sys
sys.path.append('..')
from ipfs_configs import retrieve_model_params

retrieve_model_params = retrieve_model_params('QmcFDhCvABSZnXbcFTaAN7PcoE8WavmLb7tghchyfscukJ','Z1Z1aU9zbEcwQUYxeVdETmZ0eE9wWDBpd2k4UlBmVXRrX3VBWExzS0Z0cz0=')
print(retrieve_model_params)

federation_packages = [retrieve_model_params]

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2


{'predictions': array([1, 0, 0, ..., 0, 0, 0], shape=(56962,)), 'feature_importances': array([0.0135003 , 0.0143911 , 0.01295356, 0.01568554, 0.03052201,
       0.01153662, 0.01276847, 0.02260413, 0.01374508, 0.03842016,
       0.06544812, 0.06469785, 0.1724865 , 0.01112866, 0.11649474,
       0.01063466, 0.05616613, 0.15206448, 0.03024105, 0.01225126,
       0.0141076 , 0.01991766, 0.00867408, 0.00805725, 0.01119234,
       0.00894308, 0.01826083, 0.01108162, 0.00970394, 0.01232119]), 'n_estimators': 100, 'max_features': 'sqrt', 'max_depth': -1}


In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def get_global_model(federation_packages):
    # Stack and average predictions
    all_predictions = np.stack([pkg['predictions'] for pkg in federation_packages])
    global_predictions = np.mean(all_predictions, axis=0)
    
    # Stack and average feature importances
    all_importances = np.stack([pkg['feature_importances'] for pkg in federation_packages])
    global_importances = np.mean(all_importances, axis=0)
    
    # Get averaged hyperparameters
    n_estimators = int(np.mean([pkg['n_estimators'] for pkg in federation_packages]))
    max_features = federation_packages[0]['max_features']
    avg_max_depth = np.mean([pkg['max_depth'] for pkg in federation_packages])
    max_depth = int(avg_max_depth) if avg_max_depth > 0 else None  # Use None for unlimited depth
    
    # Initialize global model
    global_model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_features=max_features,
        max_depth=max_depth
    )
    
    # Return the model along with global predictions and global importances
    return global_model, global_predictions, global_importances


In [3]:
#load dataset
import pandas as pd 
data = pd.read_csv("creditcard.csv") 

In [4]:
#separating the X and the Y values
X = data.drop(['Class'], axis = 1) 
Y = data["Class"] 
print(X.shape) 
print(Y.shape) 
# getting just the values for the sake of processing  
# (its a numpy array with no columns) 
xData = X.values 
yData = Y.values 

(284807, 30)
(284807,)


In [5]:
#training and testing data bifurcation
from sklearn.model_selection import train_test_split 
#split the data into training and testing sets 
xTrain, xTest, yTrain, yTest = train_test_split(xData, yData, test_size = 0.2, random_state = 42) 

In [6]:
# Create global model
global_model, global_predictions, global_importances = get_global_model(federation_packages)


# Train the global model with the training data
global_model.fit(xTrain, yTrain)

# Make predictions with global model
global_test_predictions = global_model.predict(xTest)

# Calculate and print performance metrics
print("\nGlobal Model Performance:")
print(f"Accuracy: {accuracy_score(yTest, global_test_predictions):.4f}")
print(f"Precision: {precision_score(yTest, global_test_predictions):.4f}")
print(f"Recall: {recall_score(yTest, global_test_predictions):.4f}")
print(f"F1 Score: {f1_score(yTest, global_test_predictions):.4f}")


Global Model Performance:
Accuracy: 0.9996
Precision: 0.9747
Recall: 0.7857
F1 Score: 0.8701
